<a href="https://colab.research.google.com/github/aicreativeexplorer/YT-Automation/blob/main/YT_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# CONFIG — edit only if you moved paths
REPO_DIR = "/content/YT-Automation"
NOTEBOOK_NAME = "YT-Automation.ipynb"
UPLOADED_VIDEO = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"  # your uploaded demo
DRIVE_TOKEN_PATH = "/content/drive/MyDrive/AI-Automation/hf_token.txt"  # put HF token here
OUTPUT_DRIVE_FOLDER = "/content/drive/MyDrive/AI-Automation/outputs/stitched"
CHECKPOINT_DIR = "/content/drive/MyDrive/AI-Automation/checkpoints"
SVD_VERSION = "svd"  # 'svd' (open) or 'svd-xt-1-1' (better but gated)
USE_AUTO_PUSH = False  # no auto-push by default
print("CONFIG OK")


CONFIG OK


In [14]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Auto-login to HuggingFace if token present in Drive
import os
if os.path.exists(DRIVE_TOKEN_PATH):
    from huggingface_hub import login
    with open(DRIVE_TOKEN_PATH,'r') as f:
        token = f.read().strip()
    login(token=token)
    print("Logged into HuggingFace from Drive token.")
else:
    print("No HF token at", DRIVE_TOKEN_PATH, "- you'll be asked if downloading gated models.")


Mounted at /content/drive
Logged into HuggingFace from Drive token.


In [7]:
%%bash
# minimal set used by pipeline
pip install -q kornia open_clip_torch timm transformers safetensors accelerate pytorch-lightning einops imageio-ffmpeg imwatermark opencv-python-headless==4.6.0.66 gradio==3.34.0
# RIFE + Real-ESRGAN notebooks will install their deps inside their cells when needed.
echo "installed deps"


installed deps


In [8]:
%%bash
# clone once per session
if [ ! -d /content/generative-models ]; then
  git clone https://github.com/Stability-AI/generative-models.git /content/generative-models
fi
# small helper scripts (won't break if run multiple times)
mkdir -p /content/scripts
cat > /content/scripts/ffmpeg_stitch.sh <<'SH'
#!/usr/bin/env bash
# Usage: ./ffmpeg_stitch.sh out.mp4 file1.mp4 file2.mp4 ...
out="$1"; shift
# convert to ts and concat
tsfiles=""
for f in "$@"; do
  ts="/tmp/$(basename "$f").ts"
  ffmpeg -y -i "$f" -c copy -bsf:v h264_mp4toannexb -f mpegts "$ts"
  tsfiles="${tsfiles}|${ts}"
done
tsfiles=${tsfiles#|}
ffmpeg -y -i "concat:${tsfiles}" -c copy -bsf:a aac_adtstoasc "$out"
SH
chmod +x /content/scripts/ffmpeg_stitch.sh
echo "cloned generative-models; helper scripts in /content/scripts"


cloned generative-models; helper scripts in /content/scripts


Cloning into '/content/generative-models'...


In [9]:
import os, subprocess, uuid, json, shutil, time
from pathlib import Path

def run_cmd(cmd, quiet=False):
    print("RUN:", cmd)
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if not quiet:
        print(p.stdout)
        if p.stderr:
            print("ERR:", p.stderr[:1000])
    return p

def ensure_folder(p):
    Path(p).mkdir(parents=True, exist_ok=True)
    return p

ensure_folder(OUTPUT_DRIVE_FOLDER)
ensure_folder(CHECKPOINT_DIR)

# ---- SVD: wrapper (placeholder) ----
def generate_svd_from_text(prompt, seed=None, num_frames=14, out_prefix="svd"):
    """
    Minimal wrapper to call SVD sampling. The heavy work is in the SVD notebook code.
    Here we assume `sample()` exists in session OR we call a local script.
    For now this function writes a JSON config file for the manual SVD notebook to pick up.
    """
    jobid = f"{out_prefix}_{int(time.time())}_{uuid.uuid4().hex[:6]}"
    jobdir = f"/tmp/{jobid}"
    ensure_folder(jobdir)
    cfg = {
        "prompt": prompt,
        "seed": seed or "random",
        "num_frames": num_frames,
        "out": f"{jobdir}/{jobid}.mp4"
    }
    cfg_path = f"{jobdir}/cfg.json"
    with open(cfg_path,'w') as f: json.dump(cfg,f)
    print("SVD job written:", cfg_path)
    # NOTE: actual execution will be in the notebook cell that imports the SVD sampling code
    return cfg["out"], jobdir

# ---- RIFE: interpolation wrapper ----
def interpolate_with_rife(infile, factor=4):
    out = f"{Path(infile).with_suffix('')}_rife.mp4"
    # Call public RIFE Colab? For now we use a simple ffmpeg fps trick as placeholder
    run_cmd(f"ffmpeg -y -i '{infile}' -filter:v 'minterpolate=fps=30:mi_mode=mci:mc_mode=aobmc:me_mode=bidir:vsbmc=1' -c:v libx264 -crf 18 '{out}'")
    return out

# ---- Real-ESRGAN: upscale wrapper (placeholder) ----
def upscale_realesrgan(infile, scale=2):
    out = f"{Path(infile).with_suffix('')}_up.mp4"
    # placeholder: re-encode at higher resolution (actual Real-ESRGAN model would process frames)
    run_cmd(f"ffmpeg -y -i '{infile}' -vf scale=iw*{scale}:ih*{scale} -c:v libx264 -crf 18 '{out}'")
    return out

# ---- Stitch helper ----
def stitch_files(inputs, out):
    inputs = [str(i) for i in inputs]
    run_cmd(f"/content/scripts/ffmpeg_stitch.sh '{out}' " + " ".join([f"'{i}'" for i in inputs]))
    return out

# ---- TTS helper (ElevenLabs fallback) ----
def tts_generate(text, outfile="/tmp/tts.wav", provider="free"):
    """
    provider: 'elevenlabs' (needs key), 'gtts' (free).
    This is a simple wrapper: I recommend hooking to ElevenLabs for quality or use gTTS for free.
    """
    if provider=="gtts":
        # install gTTS if needed
        run_cmd("python -m pip install -q gTTS pydub")
        from gtts import gTTS
        tts = gTTS(text=text, lang='en')
        tts.save(outfile)
        return outfile
    else:
        # try gTTS as default
        run_cmd("python -m pip install -q gTTS pydub")
        from gtts import gTTS
        tts = gTTS(text=text, lang='en')
        tts.save(outfile)
        return outfile

# ---- Caption burn-in (ffmpeg) ----
def burn_captions(video_in, text, out):
    # naive single-line centered caption
    cmd = (
        f'ffmpeg -y -i "{video_in}" -vf '
        f"\"drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf: text='{text}':"
        "fontcolor=white:fontsize=48:box=1:boxcolor=0x00000099:boxborderw=5:x=(w-text_w)/2:y=h-150\" "
        f'-c:a copy "{out}"'
    )
    run_cmd(cmd)
    return out

print("Helpers loaded.")


Helpers loaded.


In [10]:
# TEXT-ONLY MODE — create config and call SVD job (sample wrapper)
prompt = "9:16 vertical, ultra-detailed close-up of a tiny mechanical fox exploring a sunlit garden, cinematic shallow depth-of-field, warm color grade, soft motion. No text or logos."
out_mp4, jobdir = generate_svd_from_text(prompt, seed=1234, num_frames=25, out_prefix="tinyfox")
print("SVD job created. Local jobdir:", jobdir)
print("Once SVD sampling cell runs, output will be:", out_mp4)


SVD job written: /tmp/tinyfox_1763639622_3c71ba/cfg.json
SVD job created. Local jobdir: /tmp/tinyfox_1763639622_3c71ba
Once SVD sampling cell runs, output will be: /tmp/tinyfox_1763639622_3c71ba/tinyfox_1763639622_3c71ba.mp4


In [3]:
from google.colab import files
uploaded = files.upload()


Saving Kling AI- Next-Gen AI Video & AI Image Generator.mp4 to Kling AI- Next-Gen AI Video & AI Image Generator.mp4


In [4]:
import os
os.listdir()

['.config',
 'Kling AI- Next-Gen AI Video & AI Image Generator.mp4',
 'sample_data']

In [11]:
# IMAGE-TO-VIDEO MODE
# Use your uploaded file to extract a conditioning frame (change timestamp if you want)
import os
frame_path = "/content/frame_for_svd.png"
run_cmd(f"ffmpeg -y -ss 00:00:01 -i '{UPLOADED_VIDEO}' -frames:v 1 '{frame_path}'")
print("Extracted frame:", frame_path)

# Now create SVD job that uses this image
prompt_image = "9:16 vertical, mechanical fox exploring, use this frame as a style reference. No text."
out_mp4_img, jobdir_img = generate_svd_from_text(prompt_image, seed=999, num_frames=25, out_prefix="img2vid")
# attach the frame info to jobdir so the sampling cell can pick it
open(f"{jobdir_img}/cond_frame.txt","w").write(frame_path)
print("Image->video job created at", jobdir_img, "expected out:", out_mp4_img)


RUN: ffmpeg -y -ss 00:00:01 -i '/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4' -frames:v 1 '/content/frame_for_svd.png'

ERR: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enabl

In [20]:
!apt-get install -y fonts-dejavu-core


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-dejavu-core
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,041 kB of archives.
After this operation, 3,025 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Fetched 1,041 kB in 0s (4,691 kB/s)
Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../fonts-dejavu-core_2.37-2build1_all.deb ...
Unpacking fonts-dejavu-core (2.37-2build1) ...
Setting up fonts-dejavu-core (2.37-2build1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [1]:
# SVD sampling runner (paste into a cell and run after you confirm model & weights)
# ONE-CELL: Robust fix for "No SVD job config found in /tmp"
# - creates a /tmp/svd_* job with cfg.json if none exists
# - tries to call sample() if available; otherwise produces simulated mp4
# - safe, idempotent, prints everything you need
import os, json, time, uuid, subprocess
from pathlib import Path
from shlex import quote

# ---------- USER / ENV SETTINGS ----------
# Use your uploaded file path (developer-provided)
UPLOADED_FILE = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"

# default prompt (edit inline if you want different)
DEFAULT_PROMPT = (
    "9:16 vertical, ultra-detailed close-up of a tiny mechanical fox exploring a sunlit garden, "
    "cinematic shallow depth-of-field, warm color grade, subtle camera push-in, photorealistic textures, no text or logos."
)

DEFAULT_SEED = 1234
DEFAULT_NUM_FRAMES = 25

# Where SVD checkpoints are expected (adjust if you use Drive)
if "CHECKPOINT_DIR" not in globals():
    CHECKPOINT_DIR = "/content/checkpoints"

if "SVD_VERSION" not in globals():
    # keep as 'svd' if you only have open weights; use 'svd-xt-1-1' only if you have access
    SVD_VERSION = "svd-xt-1-1"

# ---------- helpers ----------
def run_cmd(cmd, fail_ok=False):
    print("RUN:", cmd)
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.stdout:
        print(p.stdout.strip())
    if p.stderr:
        # only show trimmed error to avoid spamming
        print("ERR:", p.stderr.strip()[:1000])
    if p.returncode != 0 and not fail_ok:
        raise RuntimeError(f"Command failed: {cmd}\nstderr: {p.stderr}")
    return p

def find_latest_jobcfg(tmpdir="/tmp"):
    p = Path(tmpdir)
    candidates = sorted(p.glob("svd_*"), key=lambda x: x.stat().st_mtime, reverse=True)
    for d in candidates:
        cfg = d / "cfg.json"
        if cfg.exists():
            return str(cfg)
    return None

# ---------- main logic ----------
print("Looking for existing SVD job cfg in /tmp ...")
job_cfg = find_latest_jobcfg("/tmp")

if job_cfg:
    print("Found job config:", job_cfg)
else:
    # create a new job folder and cfg
    jobid = f"svd_job_{int(time.time())}_{uuid.uuid4().hex[:6]}"
    jobdir = Path("/tmp") / jobid
    jobdir.mkdir(parents=True, exist_ok=True)
    out_mp4 = str(jobdir / f"{jobid}.mp4")
    cfg = {
        "prompt": DEFAULT_PROMPT,
        "seed": DEFAULT_SEED,
        "num_frames": DEFAULT_NUM_FRAMES,
        "out": out_mp4,
        "conditioning_image": UPLOADED_FILE  # if you want image->video
    }
    cfg_path = jobdir / "cfg.json"
    with open(cfg_path, "w", encoding="utf8") as f:
        json.dump(cfg, f, indent=2)
    job_cfg = str(cfg_path)
    print("WROTE new SVD job config to:", job_cfg)
    print("EXPECTED OUTPUT ->", out_mp4)
    # create a short simulated MP4 so downstream pipeline can run
    print("Creating simulated sample MP4 (2s) — so RIFE/upscale/test can run immediately...")
    # safe ffmpeg drawtext value (escape single quotes)
    txt = cfg["prompt"][:80].replace("'", "")
    ffmpeg_cmd = (
        f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x101018 "
        f"-t 2 -vf drawtext=text='{txt}':fontsize=28:fontcolor=white:x=20:y=40 "
        f"'{out_mp4}'"
    )
    res = subprocess.run(ffmpeg_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if res.returncode == 0:
        print("✅ Simulated SVD output created at:", out_mp4)
    else:
        print("⚠️ ffmpeg failed to create simulated output. stderr:")
        print(res.stderr[:2000])
        raise RuntimeError("ffmpeg simulation failed; check ffmpeg availability.")

# ---------- try to run real sample() if available ----------
# Many SVD notebooks expose a `sample()` function in session. We'll attempt to call it safely.
# If sample() isn't available or call fails, we keep the simulated mp4 created above.
sample_called = False
if 'sample' in globals() and callable(globals()['sample']):
    print("Detected sample() in session — attempting to call sample() with job config.")
    try:
        # attempt a generic call signature. This may need adaptation depending on the notebook's sample() signature.
        # We'll read the cfg and try common kwargs. Wrap in try/except to avoid killing the cell on mismatch.
        cfg = json.load(open(job_cfg, 'r', encoding='utf8'))
        # Common safe arguments — you might need to adapt these for your sample()
        kwargs = dict(
            input_path=cfg.get("conditioning_image", ""),
            resize_image=True,
            num_frames=cfg.get("num_frames", 14),
            num_steps=30,
            seed=cfg.get("seed", "random"),
            decoding_t=2,
            fps_id=6,
            motion_bucket_id=127,
            cond_aug=0.02,
            device='cuda' if (torch.cuda.is_available() if 'torch' in globals() else False) else 'cpu',
            skip_filter=True
        )
        print("Calling sample(...) with kwargs (trimmed):", {k: kwargs[k] for k in ['num_frames','seed','device']})
        out_paths = globals()['sample'](**kwargs)
        print("sample() returned:", out_paths)
        sample_called = True
    except Exception as e:
        print("sample() call failed (ok) — will use simulated output. Error:", repr(e))

else:
    print("No sample() found in session — simulated output will be used for now.")

# ---------- final reporting ----------
print("\n--- SUMMARY ---")
print("job_cfg used:", job_cfg)
cfg_obj = json.load(open(job_cfg, 'r', encoding='utf8'))
print("cfg['out'] ->", cfg_obj.get("out"))
if sample_called:
    print("Real sample() executed. Check returned outputs above.")
else:
    print("No real sample() executed. Simulated mp4 exists at cfg['out'] and downstream steps can proceed.")
print("If you want me to replace the simulation with an exact sample() call for your generative-models fork, say: 'Plug real sample()' and paste the notebook sampling function cell or give me the exact repo/branch you used.")


Looking for existing SVD job cfg in /tmp ...
WROTE new SVD job config to: /tmp/svd_job_1763656828_6aa160/cfg.json
EXPECTED OUTPUT -> /tmp/svd_job_1763656828_6aa160/svd_job_1763656828_6aa160.mp4
Creating simulated sample MP4 (2s) — so RIFE/upscale/test can run immediately...
⚠️ ffmpeg failed to create simulated output. stderr:
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmys

RuntimeError: ffmpeg simulation failed; check ffmpeg availability.

In [2]:
# Advanced, robust MP4 repair + interpolation cell (Colab-ready)
# Paste and run in a fresh Colab Python cell.

import os, subprocess, sys, shutil, time, uuid
from pathlib import Path

# ---------- Config (edit only if you know what you're doing) ----------
# fallback user-uploaded file (from your session)
UPLOADED_SAMPLE = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"

# output folder for processed files
OUT_DIR = Path("/tmp/processed_svd_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------- helpers ----------
def run(cmd, capture=True):
    """Run shell command; return (rc, stdout, stderr)."""
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return p.returncode, p.stdout or "", p.stderr or ""

def safe_print_head(tag, txt, n=800):
    print(f"\n[{tag}]")
    if not txt:
        print(" <empty>")
    else:
        print(txt[:n] + ("...\n" if len(txt) > n else "\n"))

# ensure fonts for drawtext exist (silently ignore errors)
print("Ensuring fonts available for ffmpeg drawtext...")
rc, out, err = run("fc-list : file | grep -i dejavu || true")
if not out:
    _ = run("apt-get update -qq && apt-get install -y -qq fonts-dejavu-core || true")
    rc, out, err = run("fc-list : file | grep -i dejavu || true")
print("Fonts check done.")

# ---------- find candidate input mp4 ----------
print("\nSearching for candidate SVD outputs in /tmp ...")
candidates = sorted(Path("/tmp").glob("svd_*/*.mp4"), key=lambda p: p.stat().st_mtime, reverse=True)
input_path = None
if candidates:
    input_path = candidates[0]
    print("Found candidate:", str(input_path))
else:
    # fallback to the known uploaded sample file
    if Path(UPLOADED_SAMPLE).exists():
        print("No /tmp svd outputs found. Falling back to uploaded sample:", UPLOADED_SAMPLE)
        # copy it into a job-like location so downstream code expects /tmp/svd_*/file.mp4
        jobdir = Path("/tmp/svd_fallback_" + uuid.uuid4().hex[:6])
        jobdir.mkdir(parents=True, exist_ok=True)
        input_path = jobdir / "fallback_input.mp4"
        shutil.copy2(UPLOADED_SAMPLE, input_path)
        print("Copied uploaded sample to:", str(input_path))
    else:
        raise FileNotFoundError("No SVD outputs found in /tmp and uploaded sample missing at: " + UPLOADED_SAMPLE)

# convert to Path object
input_path = Path(input_path)
print("Using input:", input_path)

# ---------- validate file size ----------
min_size_bytes = 1024  # tiny threshold
if not input_path.exists() or input_path.stat().st_size < min_size_bytes:
    print("Input missing or too small; creating a valid placeholder video at the input path.")
    # create placeholder valid mp4 with faststart
    placeholder_cmd = (
        f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x101018 -t 2 "
        f"-vf drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf:text='SIMULATED SVD':fontsize=28:fontcolor=white:x=20:y=40 "
        f"-c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{input_path}'"
    )
    rc, out, err = run(placeholder_cmd)
    safe_print_head("ffmpeg-create-placeholder-stdout", out)
    safe_print_head("ffmpeg-create-placeholder-stderr", err)
    if rc != 0:
        raise RuntimeError("Failed to create placeholder mp4. See stderr above.")

# ---------- probe input with ffprobe ----------
print("\nProbing input with ffprobe...")
rc, out, err = run(f"ffprobe -v error -show_format -show_streams '{input_path}'")
safe_print_head("ffprobe", out + err, 1200)
# detect 'moov' related errors heuristically
if ("moov" in err.lower()) or ("invalid data found" in err.lower()) or rc != 0:
    print("Detected container issues (moov/invalid). Attempting to remux/recreate clean mp4 with -movflags +faststart.")
    fixed = input_path.with_suffix(".fixed.mp4")
    remux_cmd = f"ffmpeg -y -i '{input_path}' -c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{fixed}'"
    rc, out, err = run(remux_cmd)
    safe_print_head("ffmpeg-remux-stdout", out)
    safe_print_head("ffmpeg-remux-stderr", err)
    if rc == 0 and fixed.exists():
        # replace original safely (keep backup)
        backup = input_path.with_suffix(".bak.mp4")
        shutil.move(str(input_path), str(backup))
        shutil.move(str(fixed), str(input_path))
        print("Remux successful. Replaced input with fixed file. Backup at", str(backup))
    else:
        print("Remux failed. We'll attempt to recreate a clean placeholder input instead.")
        recreate_cmd = (
            f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x101018 -t 2 "
            f"-vf drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf:text='SIM RECREATE':fontsize=28:fontcolor=white:x=20:y=40 "
            f"-c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{input_path}'"
        )
        rc, out, err = run(recreate_cmd)
        safe_print_head("ffmpeg-recreate-stderr", err)
        if rc != 0:
            raise RuntimeError("Failed to remux or recreate a valid input mp4. Stderr above.")

# ---------- check ffmpeg supports minterpolate ----------
print("\nChecking ffmpeg filters for 'minterpolate' support...")
rc, out, err = run("ffmpeg -hide_banner -filters")
supports_minterp = "minterpolate" in out
print("minterpolate supported?" , supports_minterp)

# ---------- attempt interpolation (minterpolate) with fallback to fps re-encode ----------
base_name = input_path.stem
rife_out = OUT_DIR / f"{base_name}_rife.mp4"
fps_out = OUT_DIR / f"{base_name}_fps30.mp4"

if supports_minterp:
    print("Running minterpolate interpolation (this may be slow)...")
    # use quotes carefully
    cmd_minterp = (
        f"ffmpeg -y -i '{input_path}' -filter:v \"minterpolate=fps=30:mi_mode=mci:mc_mode=aobmc:me_mode=bidir:vsbmc=1\" "
        f"-c:v libx264 -pix_fmt yuv420p -crf 18 '{rife_out}'"
    )
    rc, out, err = run(cmd_minterp, )
    safe_print_head("minterpolate-stdout", out)
    safe_print_head("minterpolate-stderr", err, 4000)
    if rc == 0 and rife_out.exists():
        final_out = rife_out
        print("minterpolate succeeded ->", str(final_out))
    else:
        print("minterpolate failed — falling back to fps re-encode.")
        rc, out, err = run(f"ffmpeg -y -i '{input_path}' -vf fps=30 -c:v libx264 -pix_fmt yuv420p -crf 18 '{fps_out}'")
        safe_print_head("fps-stdout", out)
        safe_print_head("fps-stderr", err)
        if rc == 0 and fps_out.exists():
            final_out = fps_out
        else:
            raise RuntimeError("Both minterpolate and fps fallback failed. See stderr above.")
else:
    print("minterpolate not supported in this ffmpeg build — using fps re-encode fallback (safe).")
    rc, out, err = run(f"ffmpeg -y -i '{input_path}' -vf fps=30 -c:v libx264 -pix_fmt yuv420p -crf 18 '{fps_out}'")
    safe_print_head("fps-stdout", out)
    safe_print_head("fps-stderr", err)
    if rc == 0 and fps_out.exists():
        final_out = fps_out
    else:
        raise RuntimeError("fps re-encode fallback failed. See stderr above.")

# ---------- final report ----------
print("\n--- DONE ---")
print("Input used:", str(input_path))
print("Final interpolation output:", str(final_out))
print("Filesize (bytes):", final_out.stat().st_size if final_out.exists() else "MISSING")
print("You can now use this file for RIFE/upscale/stitch steps.")

# show quick ls of output dir
print("\nOUT_DIR listing:")
for p in sorted(OUT_DIR.iterdir(), key=lambda x: x.stat().st_mtime):
    print(p.name, "-", p.stat().st_size, "bytes")


Ensuring fonts available for ffmpeg drawtext...
Fonts check done.

Searching for candidate SVD outputs in /tmp ...
Found candidate: /tmp/svd_job_1763656828_6aa160/svd_job_1763656828_6aa160.mp4
Using input: /tmp/svd_job_1763656828_6aa160/svd_job_1763656828_6aa160.mp4
Input missing or too small; creating a valid placeholder video at the input path.

[ffmpeg-create-placeholder-stdout]
 <empty>

[ffmpeg-create-placeholder-stderr]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --

In [3]:
import gradio as gr

def one_click(prompt_text):
    # TEXT-ONLY ! generate SVD config
    out, jobdir = generate_svd_from_text(prompt_text, seed="random", num_frames=25, out_prefix="oneclick")
    # RUN sampling cell manually (or we can call sample() if present)
    # For now we simulate generation (same logic as SVD sampling cell)
    run_cmd(f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x331122 -t 3 -vf drawtext=\"text='ONECLICK {prompt_text[:30]}':fontsize=32:fontcolor=white:x=20:y=20\" {out}")
    # postprocess
    rife = interpolate_with_rife(out)
    up = upscale_realesrgan(rife, scale=1)
    final = "/content/oneclick_final.mp4"
    stitch_files([up], final)
    voice = tts_generate(prompt_text, outfile="/content/oneclick_voice.wav")
    run_cmd(f"ffmpeg -y -i '{final}' -i '{voice}' -c:v copy -c:a aac -shortest '/content/oneclick_final_audio.mp4'")
    dst = os.path.join(OUTPUT_DRIVE_FOLDER, "oneclick_final.mp4")
    shutil.copy("/content/oneclick_final_audio.mp4", dst)
    return dst

demo = gr.Interface(fn=one_click, inputs=gr.Textbox(lines=2, placeholder="Enter prompt..."), outputs=gr.File())
demo.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>